<a href="https://colab.research.google.com/github/SJinji/match-tables-with-fuzzy-matching/blob/main/Reveal_Tech_Case_Jinji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install fuzzymatcher

In [3]:
! pip install recordlinkage

In [4]:
import pandas as pd
import numpy as np
import fuzzymatcher

In [5]:
import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

In [6]:
# Load the datasets into pandas DataFrames
dataset_a = pd.read_csv('dataset_A.csv', header=None)
dataset_b = pd.read_csv('dataset_B.csv', header=None)

In [11]:
# Remove column 3 from both datasets
dataset_a.drop(3, axis=1, inplace=True)
dataset_b.drop(3, axis=1, inplace=True)

# Add column names to datasets A and B
column_names = ['id', 'company_name', 'website', 'phone_number', 'address', 'postcode', 'region', 'country']
dataset_a.columns = column_names
dataset_b.columns = column_names

In [12]:
# Function to preprocess the columns
def preprocess_columns(df):
    # Convert company names to lowercase and replace "'" with whitespace
    df['company_name'] = df['company_name'].str.lower().str.replace("'", ' ')

    # Remove prefixes "http://" and "www." from the website column
    df['website'] = df['website'].str.replace(r'^https?://', '', case=False)
    df['website'] = df['website'].str.replace(r'^www\.', '', case=False)
    # Remove trailing slashes from the website column
    df['website'] = df['website'].str.rstrip('/')

    # Remove leading zeros from phone numbers
    df['phone_number'] = df['phone_number'].str.replace(r'\.', '')
    df['phone_number'] = df['phone_number'].str.replace(r'^\+?33|^\+?0*', '').str.replace(r'\s', '')

    # Remove commas from addresses
    df['address'] = df['address'].str.replace(',', '').str.lower()

    # Remove ".0" from postcode and convert it to an integer
    df['postcode'] = df['postcode'].astype(str).str.replace(r'\.0$', '')
    df['postcode'] = pd.to_numeric(df['postcode'], errors='coerce')

    # Convert region names to lowercase
    df['region'] = df['region'].str.lower()

    # Convert country names to lowercase
    df['country'] = df['country'].str.lower()

    # Remove '-' from specified columns
    columns_to_remove_dash = ['company_name', 'address', 'region', 'country']
    for col in columns_to_remove_dash:
        df[col] = df[col].str.replace('-', '').str.strip()

    # Remove duplicates from the DataFrame
    df.drop_duplicates(inplace=True)

    return df


In [13]:
# Preprocess datasets
preprocess_columns(dataset_a)
preprocess_columns(dataset_b)

,id,company_name,website,phone_number,address,postcode,region,country
0,23,groupe bertrand angelina (parent),NaN,NaN,NaN,NaN,NaN,france
1,36,sushi shop amiens,NaN,NaN,NaN,80000.0,NaN,france
2,61,sushi shop vieux lille,NaN,NaN,NaN,59000.0,NaN,france
3,87,ibis angers,NaN,NaN,NaN,41000.0,NaN,france
4,98,le christine,NaN,140517164,NaN,75006.0,NaN,france
...,...,...,...,...,...,...,...,...
8731,424609,mercure lisieux normandie,accorhotels.com,231611717,NaN,14100.0,NaN,NaN
8732,424650,inmac wstore,inmacwstore.com,NaN,NaN,NaN,NaN,NaN
8733,424662,ma french bank,mafrenchbank.fr,NaN,115 rue de sèvres,75006.0,paris6earrondissement,france
8734,424857,hôtel ibis brignolles,accorhotels.com,494691929,NaN,83170.0,NaN,NaN


In [14]:
dataset_a.columns = [f'{col}_a' for col in column_names]
dataset_b.columns = [f'{col}_b' for col in column_names]

# Fuzzy Matching

This package leverages sqlite’s full text search capability to try to match records in two different DataFrames.

In [15]:
# Find common ids based on id, website, or phone_number
common_ids_id = dataset_a.merge(dataset_b, left_on='id_a', right_on='id_b', how='inner')

# Filter out rows with non-null values in the "website_a" column
dataset_a_website_notnull = dataset_a.dropna(subset=['website_a'])
common_ids_website = dataset_a_website_notnull.merge(dataset_b, left_on='website_a', right_on='website_b', how='inner')

# Filter out rows with non-null values in the "phone_number_a" column
dataset_a_phone_notnull = dataset_a.dropna(subset=['phone_number_a'])
common_ids_phone = dataset_a_phone_notnull.merge(dataset_b, left_on='phone_number_a', right_on='phone_number_b', how='inner')

# Concatenate all common_ids based on different criteria to get unique common ids
common_ids = pd.concat([common_ids_id, common_ids_website, common_ids_phone]).drop_duplicates()

# Exclude common_ids from dataset_a and dataset_b to get the unmatched rows
dataset_a_only = dataset_a[~dataset_a['id_a'].isin(common_ids['id_a'])]
dataset_b_only = dataset_b[~dataset_b['id_b'].isin(common_ids['id_b'])]

In [16]:
# Fill NaN values with empty strings in dataset_a
dataset_a_only.fillna('', inplace=True)
# Fill NaN values with empty strings in dataset_b
dataset_b_only.fillna('', inplace=True)

In [17]:
left_on = ["company_name_a",'website_a','phone_number_a',"address_a",'postcode_a','region_a', 'country_a']
right_on = ["company_name_b",'website_b','phone_number_b',"address_b",'postcode_b','region_b', 'country_b']

In [18]:
# Running time: 1min
matched_results = fuzzymatcher.fuzzy_left_join(dataset_a_only,
                                            dataset_b_only,
                                            left_on,
                                            right_on,
                                            left_id_col='id_a',
                                            right_id_col='id_b')

In [19]:
cols = [
    "best_match_score",'id_a',"company_name_a",'website_a','phone_number_a',"address_a",'postcode_a','region_a','country_a',
    'id_b',"company_name_b",'website_b','phone_number_b',"address_b",'postcode_b','region_b','country_b'
]

# Filter out rows with NaN in 'best_match_score' column
matched_results = matched_results[matched_results['best_match_score'].notna()]

# Convert 'id_b' to integer data type
matched_results['id_b'] = matched_results['id_b'].astype(int)


# Sort the results by best_match_score in descending order
matched_results[cols].sort_values(by=['best_match_score'], ascending=False)

,best_match_score,id_a,company_name_a,website_a,phone_number_a,address_a,postcode_a,region_a,country_a,id_b,company_name_b,website_b,phone_number_b,address_b,postcode_b,region_b,country_b
164105,2.446235,250279,l unionl ardennais,lunion.fr,,21 rue des frères gilbert,7800.0,ath,belgium,70446,l unionl ardennais,journal-lunion.fr,,21 rue des frères gilbert,7800.0,ath,belgium
26734,1.921578,26832,west suffolk college,westsuffolkcollege.ac.uk,441284701301,out risbygate,,bury st edmunds,united kingdom,200405,west suffolk college,wsc.ac.uk,1284716333,out risbygate,,bury st edmunds,united kingdom
209681,1.816910,379181,talentia software,talentia-software.com,,21 rue lavoisier,92500.0,rueil malmaison,france,385229,talentia software,talenta-software.com,,21 rue lavoisier,92500.0,rueil malmaison,france
58320,1.814262,53405,sonovision group,relay1.sonovisiongroup.com,,160 avenue aristide briand,92220.0,bagneux,france,198770,sonovision group,sonovisiongroup.com,,160 avenue aristide briand,92220.0,bagneux,france
129504,1.788510,173694,allianz worldwide partners,allianz-worldwide-partners.fr/contact,,7 rue dora maar,93400.0,saintouen,france,105136,allianz partners,allianz-worldwide-partners.fr,153255325,7 rue dora maar,93400.0,saintouen,france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175302,-1.860505,266560,4 pieds groupe confort decor,4-pieds.com,299055760,parc d'activités les grandes landes 9 rue blai...,35580.0,,france,373629,4 pieds,,,,,,
138123,-1.890183,180260,barry callebaut,,,,,,,310535,metro cash & carry france,,,za du petit nanterre 5 rue des grands pr�s,92024.0,nanterre cedex,france
152952,-2.155894,234585,crédit mutuel océan,creditmutuel.fr/cmo/fr/index.html,251475300,46 rue du port boyer b.p. 92636 cedex 3,75436.0,la roche sur yon,france,175950,credit mutuel anjou,,,,,,
87387,-2.386287,81158,adversitement,"adversitement.fr, https://twitter.com/sgendrel...",(207)060-2814,18 rue de turbigo,,paris,france,251765,printemps,printemps.fr,,102 rue de provence,75009.0,paris,france


In [20]:
matched_results[cols].query("best_match_score <= -.01").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,id_a,company_name_a,website_a,phone_number_a,address_a,postcode_a,region_a,country_a,id_b,company_name_b,website_b,phone_number_b,address_b,postcode_b,region_b,country_b
5749,-0.011734,8211,bureau vallée,,,,,,,317667,bureau veritas france,,,,,,
200138,-0.011734,349794,bureau vallée,,,,,,,317667,bureau veritas france,,,,,,
38105,-0.013759,38655,segef,segef.com,534314021,2 rue kellermann,59100.0,roubaix,france,191579,ovh ltd,deftagroup.com,,2 rue kellermann,59100.0,roubaix,france
110203,-0.013946,105247,cofares,,,,,,spain,104719,vodafone,,,,,,spain
98266,-0.014239,93172,transport for london,,,,,,,180585,alstom transport,,,,,,


In [21]:
# Filter matched_results based on best_match_score >= -0.01
filtered_matched_results = matched_results[matched_results['best_match_score'] >= -0.01]

# Concatenate filtered_matched_results with common_ids
final_result = pd.concat([filtered_matched_results, common_ids], axis=0)

# Keep only the desired columns
final_result = final_result[['id_a', 'company_name_a', 'id_b', 'company_name_b', 'best_match_score']]

# Save the result as a CSV file
final_result.to_csv('matched_results.csv', index=False)


In [22]:
final_result

,id_a,company_name_a,id_b,company_name_b,best_match_score
0,186,dupont café (parent),136916,dupont café,0.158676
89,259,tempête sous un crâne,144638,tempète sous un crâne,0.349689
971,871,sushi juliette,124451,sushi juliette,0.284954
972,902,la dolce vita,119310,la dolce vita,0.607392
1331,1143,yogurt factory la défense,6067,pullman paris la défense,0.029794
...,...,...,...,...,...
6277,405677,serge blanco groupe lby,82680,serge blanco,NaN
6281,406876,sport thieme,82629,sportthieme,NaN
6284,412347,adidas us,239082,adidas group,NaN
6287,412937,repetto,241703,repetto paris,NaN
